In [2]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import sqlite3
import numpy as np

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [18]:

conn = sqlite3.connect('./data/cars.db')
sql_query = '''

      SELECT
        dealership_name, count(*)
      FROM
        inventory
      group by 1

'''
result = pd.read_sql_query(sql_query, conn)
result

,dealership_name,count(*)
0,Avon Auto Brokers,2243
1,Blasius Boston,603
2,Bostonyan Auto Group,133
3,CT Auto,247
4,Direct Auto Mecca,1023
5,Fafama Auto Sales,884
6,Irwin Automotive Group,1269
7,J&M Automotive,3374
8,Johns Auto Sales,946
9,Newton Automotive Sales,144


In [15]:

conn = sqlite3.connect('./data/cars.db')
sql_query = '''

      SELECT
        *
      FROM
        inventory
      where dealership_name = 'Stream Auto Outlet'

'''
result = pd.read_sql_query(sql_query, conn)
result

,title,year,make,model_trim,vehicle_type,model,trim,vehicle_mileage,price,exterior_color,interior_color,transmission,engine,drivetrain,vin,dealership_name,dealership_address,dealership_zipcode,dealership_city,dealership_state,inventory_url,scraped_date
0,2021 Acura ILX Sedan w/Premium Package,2021,Acura,ILX w/Premium Package,Sedan,ILX,w/Premium Package,6078.0,27490,Performance Red Pearl,Black,Automatic,Premium Unleaded I-4 2.4 L/144,FWD,19UDE2F70MA011357,Stream Auto Outlet,324 W Merrick Rd,11580,Valley Stream,NY,https://www.streamautooutlet.com/inventory?typ...,2022-09-04 03:23:49.483983
1,2020 Acura ILX Sedan,2020,Acura,ILX Base,Sedan,ILX,Base,30384.0,22990,Gray,Black,Automatic,Premium Unleaded I-4 2.4 L/144,FWD,19UDE2F34LA004973,Stream Auto Outlet,324 W Merrick Rd,11580,Valley Stream,NY,https://www.streamautooutlet.com/inventory?typ...,2022-09-04 03:23:49.483983
2,2020 Acura MDX SH-AWD 7-Passenger,2020,Acura,MDX 3.5L,SUV,MDX,3.5L,6332.0,37990,Red,Beige,Automatic,Premium Unleaded V-6 3.5 L/212,AWD,5J8YD4H30LL004177,Stream Auto Outlet,324 W Merrick Rd,11580,Valley Stream,NY,https://www.streamautooutlet.com/inventory?typ...,2022-09-04 03:23:49.483983
3,2019 Acura MDX SH-AWD w/Technology/A-Spec Pkg,2019,Acura,MDX w/Technology/A-Spec Pkg,SUV,MDX,w/Technology/A-Spec Pkg,34422.0,37990,Blue Pearl,Red,Automatic,Premium Unleaded V-6 3.5 L/212,AWD,5J8YD4H06KL021182,Stream Auto Outlet,324 W Merrick Rd,11580,Valley Stream,NY,https://www.streamautooutlet.com/inventory?typ...,2022-09-04 03:23:49.483983
4,2020 Acura MDX SH-AWD 7-Passenger w/Technology...,2020,Acura,MDX w/Technology Pkg,SUV,MDX,w/Technology Pkg,15703.0,39990,White,None,Automatic,Premium Unleaded V-6 3.5 L/212,AWD,5J8YD4H56LL033894,Stream Auto Outlet,324 W Merrick Rd,11580,Valley Stream,NY,https://www.streamautooutlet.com/inventory?typ...,2022-09-04 03:23:49.483983
5,2020 Acura RDX SH-AWD w/Technology Pkg,2020,Acura,RDX w/Technology Pkg,SUV,RDX,w/Technology Pkg,34116.0,34490,Black,Black,Automatic,Intercooled Turbo Premium Unleaded I-4 2.0 L/122,AWD,5J8TC2H50LL007082,Stream Auto Outlet,324 W Merrick Rd,11580,Valley Stream,NY,https://www.streamautooutlet.com/inventory?typ...,2022-09-04 03:23:49.483983
6,2019 Acura TLX 3.5L SH-AWD w/Technology Pkg,2019,Acura,TLX w/Technology Pkg,Sedan,TLX,w/Technology Pkg,33646.0,28990,Silver,Black,Automatic,Premium Unleaded V-6 3.5 L/212,AWD,19UUB3F52KA801723,Stream Auto Outlet,324 W Merrick Rd,11580,Valley Stream,NY,https://www.streamautooutlet.com/inventory?typ...,2022-09-04 03:23:49.483983
7,2020 Acura TLX 3.5L FWD,2020,Acura,TLX 3.5L V6,Sedan,TLX,3.5L V6,52756.0,24990,Gray,Black,Automatic,Premium Unleaded V-6 3.5 L/212,FWD,19UUB2F31LA001585,Stream Auto Outlet,324 W Merrick Rd,11580,Valley Stream,NY,https://www.streamautooutlet.com/inventory?typ...,2022-09-04 03:23:49.483983
8,2021 Acura TLX FWD w/Technology Package,2021,Acura,TLX w/Technology Package,Sedan,TLX,w/Technology Package,12371.0,37490,Violet,Black,Automatic,Intercooled Turbo Premium Unleaded I-4 2.0 L/122,FWD,19UUB5F44MA013144,Stream Auto Outlet,324 W Merrick Rd,11580,Valley Stream,NY,https://www.streamautooutlet.com/inventory?typ...,2022-09-04 03:23:49.483983
9,2020 Acura TLX 3.5L SH-AWD w/Technology Pkg,2020,Acura,TLX w/Technology Pkg,Sedan,TLX,w/Technology Pkg,38556.0,30990,Platinum White Pearl,None,Automatic,Premium Unleaded V-6 3.5 L/212,AWD,19UUB3F44LA001580,Stream Auto Outlet,324 W Merrick Rd,11580,Valley Stream,NY,https://www.streamautooutlet.com/inventory?typ...,2022-09-04 03:23:49.483983


In [13]:

conn = sqlite3.connect('./data/cars.db')
sql_query = '''

      SELECT
        *
      FROM
        parsing_errors

'''
# result = pd.read_sql_query(sql_query, conn)
# result

In [21]:

conn = sqlite3.connect('./data/cars.db')
sql_query = '''

      SELECT
        date(scraped_date, 'start of month') as start_month, count(*)
      FROM
        inventory
      group by 1

'''
result = pd.read_sql_query(sql_query, conn)
result

,start_month,count(*)
0,2022-03-01,1546
1,2022-04-01,1351
2,2022-05-01,3295
3,2022-07-01,1598
4,2022-08-01,1545
5,2022-09-01,1725


In [4]:
# ## Targeted drop
# TABLE_NAME = 'inventory'

# #Connecting to sqlite
# conn = sqlite3.connect('./data/cars.db')

# #Creating a cursor object using the cursor() method
# cursor = conn.cursor()

# #Doping EMPLOYEE table if already exists
# cursor.execute(f"DELETE FROM {TABLE_NAME} WHERE make is null")

# #Commit your changes in the database
# conn.commit()

# #Closing the connection
# conn.close()